<a href="https://colab.research.google.com/github/21092004Goda/data_anal/blob/main/Lab_1_Computer_Vision_and_Pattern_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Лабораторная работа 1.**
#### *Сбор, визуализация и анализ данных по графу*

### **Выбор тематики научных публикаций и сбор**
В качестве тематики для анализа был выбрана область Computer Vision and Pattern Recognition *(CVPR)*. Данное направление является одним из наиболее динамично развивающихся в сфере искусственного интеллекта и машинного обучения, что обусловлено его широким практическим применением: от автономных транспортных систем и медицинской диагностики до обработки изображений и видеоаналитики.

### **Сбор данных о научных публикациях через arXiv API**

**Область:** *Computer Vision and Pattern Recognition*

**Источник:** [arxiv.org](https://arxiv.org/list/cs.CV/recent)


In [ ]:
%%capture
!pip install feedparser


In [ ]:
import feedparser
import urllib.request as libreq
import pandas as pd


In [ ]:
# Параметры запроса
base_url = 'http://export.arxiv.org/api/query?'
search_query = 'cat:cs.CV'  # Категория: Computer Vision
start = 0                   # Начальный индекс
max_results = 200           # Количество результатов за запрос (максимум 2000)


In [ ]:
# Формирование URL
url = f"{base_url}search_query={search_query}&start={start}&max_results={max_results}"


In [ ]:
# Выполнение запроса и разбор ответа
with libreq.urlopen(url) as response:
    data = response.read()
feed = feedparser.parse(data)


In [ ]:
# Извлечение нужных данных
articles = []
for entry in feed.entries:
    authors = ', '.join(author.name for author in entry.authors)
    tags = [tag['term'] for tag in entry.get('tags', [])]
    articles.append({
        'title': entry.title,
        'abstract': entry.summary,
        'authors': authors,
        'tags': ", ".join(tags),
        'published': entry.published,
        'arxiv_url': entry.link
    })



In [ ]:
# Создание DataFrame
df = pd.DataFrame(articles)
print(df.shape)
df.head()

### **Загрузка модели для извлечения ключевых слов с HuggingFace**

**Модель:** [HuggingFace](https://huggingface.co/ilsilfverskiold/tech-keywords-extractor)

In [ ]:
# 2. Извлечение ключевых слов
from transformers import pipeline
from tqdm.auto import tqdm
import torch

print(torch.cuda.is_available())  # True = GPU доступен
print(torch.cuda.get_device_name(0))  # имя GPU

pipe = pipeline("text2text-generation", model="ilsilfverskiold/tech-keywords-extractor", device=0)

tqdm.pandas()
df["keywords"] = df["abstract"].progress_apply(lambda x: pipe(x)[0]["generated_text"].split(", "))

df.head()

In [ ]:
# 3. Сохранение датасета
df.to_csv("cvpr_pubs.csv", index=False)

In [ ]:
# 4. Построение графа по ключевым словам
import networkx as nx
from itertools import combinations, chain
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

klist = [[word.strip() for word in keywords if word] for keywords in df["keywords"].tolist()]
edges = list(chain(*[list(combinations(words, 2)) for words in klist if len(words) > 1]))
edges = [tuple(sorted(edge)) for edge in edges]
weighted_edges = [(edge[0], edge[1], {"weight": edges.count(edge)}) for edge in set(edges)]

G = nx.Graph()
G.add_edges_from(weighted_edges)

plt.figure(figsize=(12, 8))
nx.draw(G, with_labels=False, node_size=20)
plt.show()


In [ ]:
# 5. Кластеры (поиск сообществ)
communities = nx.community.louvain_communities(G, resolution=0.9)
print(f"Clusters: {len(communities)}")


Clusters: 101


In [ ]:
# 6. Топовые узлы по центральности
from operator import itemgetter

for i, nodes in enumerate(communities[:3], start=1):
    s = G.subgraph(nodes)
    top_nodes = sorted(nx.degree_centrality(s).items(), key=itemgetter(1), reverse=True)[:5]
    print(f"Cluster {i}, top nodes: {top_nodes}")
